In [9]:
import numpy as np

In [199]:
def stat(T):
    w, v = np.linalg.eig(T.T)
    
    j_stat =np.argmin(abs(w-1.0))
    p_stat=v[:,j_stat].real
    p_stat /= p_stat.sum()
    return p_stat

In [126]:
T=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(T)
a=T[:,[0,2]]
print(a)

print(a[[0,2],:])

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 3]
 [4 6]
 [7 9]]
[[1 3]
 [7 9]]


In [131]:
T=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(T[:,[0,2]])
c=(T[:,[0,2]]).sum(axis=1)
c= -1*c
c

[[1 3]
 [4 6]
 [7 9]]


array([ -4, -10, -16])

In [139]:
h=np.ones(10)
h[[1,4,5]]=np.array([9,4,5])
h

array([ 1.,  9.,  1.,  1.,  4.,  5.,  1.,  1.,  1.,  1.])

In [162]:
def hitting_probability(T,S):
    """
    We calculate the hitting time of a set S, defined 
    
    INPUT:
        - T = A markov transition matrix. Square matrix.
        - S = Subset of states.
        
    OUTPUT:
        
    
    """
    
    n = len(T)
    
    h=np.zeros(n)
    h[S] = np.ones(len(S))
    
    S_ = [i for i in range(n) if i not in S]
    
    A = np.copy(T)
    A = A[:,S_]
    A = A[S_,:]
    
    I=np.identity(len(A))
    A=A-I
    
    B = np.copy(T)
    B = B[:,S]
    B = B[S_,:]

    b=B.sum(axis=1)
    b= -1*b
    
    Q = (np.linalg.lstsq(A,b))[0]
    h[S_] = Q
    
    return h
        

In [163]:
T=np.array([[0.9,0.1,0,0],[0.1,0.89,0.01,0],[0,0.01,0.79,0.2],[0,0,0.2,0.8]])
X=[1,2]
hitting_probability(T,X)

array([ 1.,  1.,  1.,  1.])

In [161]:
n=10
A=[1,2,4]
B=[6,8,9]
AB_ = [i for i in range(n) if i not in A and i not in B]
AB_
h=np.ones(n)
h[B] = np.zeros(len(B))
h

array([ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.])

In [250]:
def forward_commitors(T,A,B):
    """
    Formally defined as the probability to hit set B rather than A
    
    INPUT:
        - T = A markov transition matrix
        - A = A subset of states.
        - B = A subset of states.
        
    OUTPUT:
    """

        
    n = len(T)
    
    q=np.zeros(n)
    q[B] = np.ones(len(B))
    
    B_ = [i for i in range(n) if i not in B]
    AB_ = [i for i in B_ if i not in A]
    
    
    a = np.copy(T)
    a = a[:,AB_]
    a = a[AB_,:]
    
    I=np.identity(len(a))
    a=a-I
    
    b = np.copy(T)
    b = b[:,B]
    b = b[AB_,:]

    b=b.sum(axis=1)
    b= -1*b 
    
    if not(len(a)==0 or len(b)==0):
        Q = (np.linalg.lstsq(a,b))[0]
        h[AB_] = Q
    
    return q        

In [251]:
T

array([[ 0.9 ,  0.1 ,  0.  ,  0.  ],
       [ 0.1 ,  0.89,  0.01,  0.  ],
       [ 0.  ,  0.01,  0.79,  0.2 ],
       [ 0.  ,  0.  ,  0.2 ,  0.8 ]])

In [252]:
A=[0,1]
B=[2,3]

forward_commitors(T,A,B)

array([ 0.,  0.,  1.,  1.])

In [231]:
s=np.array([[1,2,4],[3,4,5],[4,6,7]])
print(s)
s[1,2]

[[1 2 4]
 [3 4 5]
 [4 6 7]]


5

In [253]:
def probability_current(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    The probability current gives the averege number of reactive trajectories
    (those going from A to B, without entering A before reaching B) that transition
    from i to j per time unit.
    
    """
    
    n = len(T)
    
    F = np.copy(T)
    
    q = forward_commitors(T,A,B)   
    q_ = forward_commitors(T,B,A)
    
    pi=stat(T)
    
    for i in range(n):
        for j in range(n):
            if j==i:
                F[i,j]=0
            else:
                F[i,j] = (pi[i])*(q_[i])*(T[i,j])*(q[j])
    
    return F

In [254]:
A=[0,1]
B=[2,3]
probability_current(T,A,B)

array([[ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.0025,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [ ]:
def effective_probability_current(T,A,B):
    """
    
    
    """

In [267]:
def average_current(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    Average total number of trajectories going from A to B per time unit.
    
    INPUT:
        - T = A markov transition matrix.
        - A = A subset of indices.
        - B = A subset of indices.
    
    """
    
    f_AB = probability_current(T,A,B)
        
    n=len(f_AB)
    F_AB = np.zeros([n,n])
    
    for i in A:
        for j in B:
            F_AB[i,j] = f_AB[i,j]
    F_AB=F_AB.sum(axis=1)
    F_AB=F_AB.sum(axis=0)
    
    return F_AB

In [268]:
A=[0,1]
B=[2,3]
print(probability_current(T,A,B))
average_current(T,A,B)

[[ 0.      0.      0.      0.    ]
 [ 0.      0.      0.0025  0.    ]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]]


0.0025000000000000955

In [257]:
s=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(s)
print(s.sum(axis=1))
K=s.sum(axis=1)
print(s.sum(axis=0))
K.sum(axis=0)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[ 6 15 24]
[12 15 18]


45

In [284]:
def transition_rates(T,A,B):
    """
    ¡¡NOT OPTIMIZED!!
    
    The forward and backward commitor are sufficient to calculate transition rates between 
    the sets A and B
    
    """
    
    F_AB = average_current(T,A,B)
    pi = stat(T)
    q = forward_commitors(T,A,B)
    
    sum=0
    
    for j in B:
        sum= sum + (pi[j])*(q[j])
        
    return (F_AB/sum)
    

In [285]:
A=[0,1]
B=[2,3]
transition_rates(T,A,B)

0.005000000000000393

In [288]:
def mean_first_passage_times(T,A,B):
    """
    
    """
    
    t= transition_rates(T,A,B)
    return 1/t

In [289]:
A=[0]
B=[1]
mean_first_passage_times(T,A,B)

9.9999999999999556